<a href="https://colab.research.google.com/github/JTaal/bioinformatic_scripts/blob/main/healthcare_random_forest_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')
from numerize import numerize
import plotly.express as px


pd.set_option('display.max_columns',None)

In [ ]:
#Unzip file from zipfile

from zipfile import ZipFile
import zipfile

filepath = os.getcwd()
with zipfile.ZipFile('healthcare_dataset.csv.zip','r') as zip_ref:
    zip_ref.extractall(filepath)

In [ ]:
df = pd.read_csv('healthcare_dataset.csv')

In [ ]:
df.head()

In [ ]:
#Remove all unwanted white space from dataset
df = df.replace(r'\s+', ' ', regex=True)

In [ ]:
#Change the name of columns
df.columns = df.columns.str.replace(" ","_")

In [ ]:
#Checking null values percentage
print("==="*25)
print("Checking Data has null values or not")
print("==="*25)
print(round(df.isnull().sum() * 100 / len(df),2))

In [ ]:
#Information
print("Data Information")
print("==="*25)
print(df.info())

- Info looks good

In [ ]:
df.describe().T

In [ ]:
df.describe(include=object).T

- Using describe we can retrieve lot of information

In [ ]:
#Dividing dataset into numerical columns and categorical columns
df1=df.copy()
#Separating numerical and categorical columns
num=[]
cat=[]
for col in df1.columns:
    if pd.api.types.is_numeric_dtype(df1[col]):
        num.append(col)
    elif pd.api.types.is_object_dtype(df1[col]):
        cat.append(col)

print('Numerical columns : ',num)
print('Categorical columns : ',cat)

In [ ]:
# Calculate the correlation matrix
numerical_columns = df.select_dtypes(include=['int64','float64']).corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(20, 6))
sns.heatmap(numerical_columns, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

- Correlation matrix do not show much of the infomation for plotted dataset
- It some what show features are negatively correlated

In [ ]:
#Numerical column analysis
for i in num:
    fig,axes=plt.subplots(1,2,figsize=(20,6))

    sns.histplot(df1[i],kde=True,ax=axes[0])
    axes[0].set_title(f'Distribution of {i}')
    axes[0].grid(True)
    axes[1].set_xlabel('')

    sns.boxplot(x=df1[i],ax=axes[1])
    axes[1].set_title(f'Box plot of {i}')
    axes[1].grid(True)
    axes[1].set_xlabel('')

    plt.tight_layout()
    plt.show()

- we can see as we don't have outliers and data is normally distributed

In [ ]:
df.columns

EDA

Univariate

In [ ]:
#Hist plot for Quantity
plt.figure(figsize=(20,6))
sns.histplot(data=df['Age'],kde=True)
plt.title('Distibution of Age')
plt.xticks(rotation=10)
plt.show()

- Most of the age is distributed around 20 and also at 80
- This shows younger age people's are more aware & concern with health-care rather then people's who's age lies in range of 21 - 65

In [ ]:
#Gender count plot

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Gender", ax = ax,order=df['Gender'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Gender", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- This graphs shows there are more females involved in heath care as this can be why women's live longer than men's(count is halfly near and difficult to compare)

In [ ]:
#Blood_Type

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Blood_Type", ax = ax,order=df['Blood_Type'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Blood_Type", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- AB is most counted blood group in the dataset followed up with AB+, B-, O+
- Type AB-negative blood is considered a universal plasma donor, meaning anyone can receive type AB-negative plasma

In [ ]:
#Blood_Type

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Medical_Condition", ax = ax,order=df['Medical_Condition'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Medical_Condition", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Most medical-condition is ASTHMA, CANCER, HYPERTENSION
- This shows most of the test are taken from most stressfull people segment and we can also sale them health benefits and sale health-insurance.

In [ ]:
#Doctor

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, y = "Doctor", ax = ax,order=df['Doctor'].value_counts().iloc[:20].index)
ax.set_title("Doctor", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Most counted doctor is *michael johnson*

In [ ]:
#Hospital

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, y = "Hospital", ax = ax,order=df['Hospital'].value_counts().iloc[:20].index)
ax.set_title("Hospital", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Most frequent hospital in dataset is SMITH PLC,  and follow up by SMITH AND SONS, SMITH LTD, SMITH INC

In [ ]:
#Insurance_Provider

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Insurance_Provider", ax = ax,order=df['Insurance_Provider'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Insurance_Provider", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Who have buyed cigna health-care are getting ill most.

In [ ]:
#Room_Number

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Room_Number", ax = ax,order=df['Room_Number'].value_counts().iloc[:20].index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Room_Number", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Admission_Type

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Admission_Type", ax = ax,order=df['Admission_Type'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Admission_Type", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Most cases are marked as urgent

In [ ]:
#Medication

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Medication", ax = ax,order=df['Medication'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Medication", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- Penicillin is a medication used to manage and treat a wide range of infections. It is in the beta-lactam antibiotic class of drugs. This activity describes penicillin's indications, action, and contraindications as a valuable agent in treating infection.

In [ ]:
#Test_Results

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, x = "Test_Results", ax = ax,order=df['Test_Results'].value_counts().index)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Test_Results", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Bi-variate

In [ ]:
df['Date_of_Admission'] = pd.to_datetime(df['Date_of_Admission'])
df['Discharge_Date'] = pd.to_datetime(df['Discharge_Date'])

In [ ]:
df['Admission_datee'] = df['Date_of_Admission'].dt.weekday
df['Admission_month'] = df['Date_of_Admission'].dt.month
df['Days_hospitalized'] = df['Discharge_Date'] - df['Date_of_Admission']
df['day_of_week'] = df['Date_of_Admission'].dt.day_of_week
df['Discharge_month'] = df['Discharge_Date'].dt.day_of_week
df['Discharge_Datee'] = df['Discharge_Date'].dt.weekday


In [ ]:
df.head()

In [ ]:
# Billing Amount according to Medical Condition and Test Results
df1 = df.groupby(['Blood_Type', 'Medical_Condition'])[['Billing_Amount']].sum().reset_index()
plt.figure(figsize=(20,10))
sns.barplot(x=df1['Medical_Condition'], y=df1['Billing_Amount'], hue=df['Test_Results'], palette="rainbow")
plt.title("Billing Amount according to Blood_Type and Medical conditions and how does cost affects on this segments")
plt.legend(loc='upper right')
plt.ylabel("Billing Amount")
plt.xticks(rotation=0, fontsize = 10)
plt.show()

In [ ]:
#Blood_Type - Medical_Conditions

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Blood_Type", hue=df['Medical_Condition'],ax = ax,order=df['Blood_Type'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Blood_Type - Medical_Conditions", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- In *AB-* we can view that most of *patient* are in Hypertention or obesity as both are connected if you are in obesity then most probably you can be in hypertention
- In *AB+* most *patient* are target of *Arthritis* and *Obesity*
- In *B-* most of *patient* are target of *Cancer* & *Diabetes*
- In *O+* most of *patient* are target of *Asthma*


In [ ]:
#Blood_Type - Gender

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Blood_Type", hue=df['Gender'],ax = ax,order=df['Blood_Type'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Blood_Type - Gender", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Blood_Type - Medication

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Blood_Type", hue=df['Medication'],ax = ax,order=df['Blood_Type'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Blood_Type - Medication", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Medical_Condition - Gender

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Medical_Condition", hue=df['Gender'],ax = ax,order=df['Medical_Condition'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Medical_Condition - Gender", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Medical_Condition - Insurance_Provider

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Medical_Condition", hue=df['Insurance_Provider'],ax = ax,order=df['Medical_Condition'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Medical_Condition - Insurance_Provider", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

In [ ]:
#Medical_Condition - Medication

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Medical_Condition", hue=df['Medication'],ax = ax,order=df['Medical_Condition'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Medical_Condition - Medication", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- In *Arthritis* most repeated *medication* is *Penicillin*
- In *Asthma* most repeated *medication* is *Aspirin*

In [ ]:
#Gender - Insurance_Provider

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Gender", hue=df['Insurance_Provider'],ax = ax,order=df['Gender'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Gender - Insurance_Provider", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- In female segment best facility is provided by Medicore and in Male segments it is totally different, in male Blue Cross is most famous

In [ ]:
#Gender - Medication

fig,ax = plt.subplots(figsize = (20,6))
sns.countplot(df, x = "Gender", hue=df['Medication'],ax = ax,order=df['Gender'].value_counts().index)
plt.legend(loc='lower right')
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(text = f"{numerize.numerize(height)}",
               xy = (rect.get_x()+rect.get_width()/2, height),
               xytext = (0,3),
               textcoords = "offset points",
               ha = "center",
               va = "bottom",
               color = "black")
ax.set_title("Gender - Medication", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

- In this graph we can see that male are having highest *medication* of *Penicillin* maybe most of male will be having *Arthritis*

In [ ]:
#Days_Hospitalized - Medical_Conditions

fig,ax = plt.subplots(figsize = (20,10))
sns.countplot(df, y = "Days_hospitalized", hue=df['Medical_Condition'],ax = ax,order=df['Days_hospitalized'].value_counts().iloc[:10].index)
plt.legend(loc='lower right')
ax.set_title("Blood_Type - Medical_Conditions", fontsize = 10, fontweight = "bold", color = "darkred")
fig.show()

**Model Building**

In [ ]:
##Splitting data into X & y

X = df.drop(columns=['Name','Date_of_Admission','Billing_Amount', 'Discharge_Date','Test_Results'],axis=1)
y = df['Test_Results']

In [ ]:
# Do Label encoding on y label

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)

In [ ]:
# Import category encoder and perfom on X data

import category_encoders as ce
X = ce.TargetEncoder(smoothing=0).fit_transform(X,y)

In [ ]:
#Train-test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape)


In [ ]:
#Standarscaler

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random = RandomForestClassifier()
random.fit(X_train,y_train)


In [ ]:
pred_2 = random.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score (y_test, pred_2)
print("Acc: %.2f%%" % (acc * 100.0))